08 Extract table 6.1 codes

Extract participants and codes for table 6.1 (CRC screening procedures)

In [1]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "WGS table 6.1 codes" for domain "procedure" and was generated for All of Us Controlled Tier Dataset v5
dataset_94718537_procedure_sql <- paste("
    SELECT
        procedure.person_id,
        procedure.procedure_concept_id,
        p_standard_concept.concept_name as standard_concept_name,
        p_standard_concept.concept_code as standard_concept_code,
        p_standard_concept.vocabulary_id as standard_vocabulary,
        procedure.procedure_datetime,
        procedure.procedure_type_concept_id,
        p_type.concept_name as procedure_type_concept_name,
        procedure.modifier_concept_id,
        p_modifier.concept_name as modifier_concept_name,
        procedure.quantity,
        procedure.visit_occurrence_id,
        p_visit.concept_name as visit_occurrence_concept_name,
        procedure.procedure_source_value,
        procedure.procedure_source_concept_id,
        p_source_concept.concept_name as source_concept_name,
        p_source_concept.concept_code as source_concept_code,
        p_source_concept.vocabulary_id as source_vocabulary,
        procedure.qualifier_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `procedure_occurrence` procedure 
        WHERE
            (
                procedure_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    2109111
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        ) 
                        OR  procedure_source_concept_id IN  (
                            SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (
                                    select
                                        cast(cr.id as string) as id 
                                    FROM
                                        `cb_criteria` cr 
                                    WHERE
                                        concept_id IN (
                                            2617223, 2617207, 2002728, 2109180, 2747302, 2109190, 2747304, 2747034, 2746233, 2002702, 2749131, 2745726, 2109188, 2745732, 2109193, 2109173, 2109200, 2109183, 2002705, 43016471, 2002704, 2109205, 2109111, 2109199, 2109171, 2109201, 2109184, 2109112, 2617208, 2747037, 2109181, 2109204, 2109198, 2109194, 43016456, 2747038, 43016447, 2109174, 2747067, 2109196, 2109113, 2109177, 2747063, 2109187, 2002703, 2109114
                                        ) 
                                        AND full_text LIKE '%_rank1]%'
                                ) a 
                                    ON (
                                        c.path LIKE CONCAT('%.',
                                    a.id,
                                    '.%') 
                                    OR c.path LIKE CONCAT('%.',
                                    a.id) 
                                    OR c.path LIKE CONCAT(a.id,
                                    '.%') 
                                    OR c.path = a.id) 
                                WHERE
                                    is_standard = 0 
                                    AND is_selectable = 1
                                )
                        )  
                        AND (
                            procedure.PERSON_ID IN (
                                SELECT
                                    distinct person_id  
                                FROM
                                    `cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    ) 
                                )
                        )
                    ) procedure 
                LEFT JOIN
                    `concept` p_standard_concept 
                        ON procedure.procedure_concept_id = p_standard_concept.concept_id 
                LEFT JOIN
                    `concept` p_type 
                        ON procedure.procedure_type_concept_id = p_type.concept_id 
                LEFT JOIN
                    `concept` p_modifier 
                        ON procedure.modifier_concept_id = p_modifier.concept_id 
                LEFT JOIN
                    `visit_occurrence` v 
                        ON procedure.visit_occurrence_id = v.visit_occurrence_id 
                LEFT JOIN
                    `concept` p_visit 
                        ON v.visit_concept_id = p_visit.concept_id 
                LEFT JOIN
                    `concept` p_source_concept 
                        ON procedure.procedure_source_concept_id = p_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
procedure_94718537_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "procedure_94718537",
  "procedure_94718537_*.csv")
message(str_glue('The data will be written to {procedure_94718537_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_94718537_procedure_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  procedure_94718537_path,
  destination_format = "CSV")



Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

The data will be written to gs://fc-secure-329f909f-3a71-416a-9c18-95db1c1f801d/bq_exports/earosenthal@preprod.researchallofus.org/20220526/procedure_94718537/procedure_94718537_*.csv. Use this path when reading the data into your notebooks in the future.



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {procedure_94718537_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- NULL
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_94718537_procedure_df <- read_bq_export_from_workspace_bucket(procedure_94718537_path)

dim(dataset_94718537_procedure_df)

head(dataset_94718537_procedure_df, 5)